In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
#把下一个新版本的特性导入到当前版本
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [3]:
url="http://mattmahoney.net/dc/"
def maybe_download(filename,expected_bytes):
    # 检查最终文件字节数和expected是否相同
    if not os.path.exists(filename):
        filename,_=urllib.request.urlretrieve(url+filename,filename)
    statinfo=os.stat(filename)
    if statinfo.st_size==expected_bytes:
        print('Found and verified',filename)
    else:
        print(statinfo.sz_size)
        raise Exception('Failed to verify'+filename+'. Can you get to it with a browser?')
    return filename
filename=maybe_download('text8 .zip',31344016)

Found and verified text8 .zip


In [4]:
# 将语料库解压，并转换成一个word的list
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data=tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
vocabulary=read_data(filename)
print('Data size',len(vocabulary))
print(vocabulary[0:100])

Data size 17005207
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']


In [5]:
#制作一个词表，，将单词映射为一个id，词表的大小为5W
#即只考虑常用的5W个单词
#将不常见的词变成一个UNK标识，统一到一个id
vocabulary_size=50000
def build_dataset(words,n_words):
    #将原始的单词变成单词
    count=[['UNK',-1]]
    count.extend(collections.Counter(words).most_common(n_words-1))
    dictionary=dict()
    for word,_ in count:
        dictionary[word]=len(dictionary)
    data=list()
    unk_count=0
    for word in words:
        if word in dictionary:
            index=dictionary[word]
        else:
            index=0 #unk的index为0
            unk_count+=1
        data.append(index)
    count[0][1]=unk_count
    reversed_dictionary=dict(zip(dictionary.values(),dictionary.keys()))
    return data,count,dictionary,reversed_dictionary

In [6]:
data,count,dictionary,reverse_dictionary=build_dataset(vocabulary,vocabulary_size)
del vocabulary #删除已节省的内存
print('Most comman words',count[:5])
print('Sample data',data[:10],[reverse_dictionary[i] for i in data[:10]])
data_index=0

Most comman words [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [7]:
#定义一个函数用于skip-gram模型用的batch
def generate_batch(batch_size,num_skips,skip_window):
    # data_index 相当于一个指针，初始化为0，每次生成一个batch，data_index会相应地往后推
    global data_index
    assert batch_size%num_skips==0
    assert num_skips<=2*skip_window
    batch=np.ndarray(shape=(batch_size),dtype=np.int32)
    labels=np.ndarray(shape=(batch_size,1),dtype=np.int32)
    span=2*skip_window+1
    buffer=collections.deque(maxlen=span)
    #data index 是当前数据的起始位置
    #产生batch往后推一位
    for _ in range(span):
        buffer.append(data[data_index])
        data_index=(data_index+1)%len(data)
    for i in range(batch_size//num_skips):
        #利用buffer生成batch，buffer是一个长度为2×skip Window + 1长度的word list
        #一个buffer生成num skips个数的样本
        target=skip_window #target label at the center of the buffer
        #target_to_avoid保证样本不重复
        targets_to_avoid=[skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target=random.randint(0,span-1)
            targets_to_avoid.append(target)
            batch[i*num_skips+j]=buffer[skip_window]
            labels[i*num_skips+j]=buffer[target]
        buffer.append(data[data_index])
        #每利用buffer生成num skips个样本，data index往后推一位
        data_index=(data_index+1)%len(data)
    data_index=(data_index+len(data)-span)%len(data)
    return batch,labels
#默认情况下skip Window=1，num skip=2
#此时是从连续的3(3=skip_window+1)个词中生成2(num_skips)个样本
batch,labels=generate_batch(batch_size=8,num_skips=2,skip_window=1)
for i in range(8):
    print(batch[i],reverse_dictionary[batch[i]],'->',labels[i,0],reverse_dictionary[labels[i,0]])

3084 originated -> 5239 anarchism
3084 originated -> 12 as
12 as -> 3084 originated
12 as -> 6 a
6 a -> 195 term
6 a -> 12 as
195 term -> 2 of
195 term -> 6 a


In [13]:
#建模
batch_size=128
embedding_size=128
skip_window=1
num_skips=2
#在模型训练时会对模型进行验证，验证的方法是找出和某个词最近的词
#只对前valid window的词进行验证
valid_size=16 #每次验证16个词
valid_window=100 #这16个词来自100个最常见的词
valid_examples=np.random.choice(valid_window,valid_size,replace=False)
#构造损失时选取的噪声词的数量
num_sampled=64
graph=tf.Graph()
with graph.as_default():
    #输入batch
    train_inputs=tf.placeholder(tf.int32,shape=[batch_size])
    train_labels=tf.placeholder(tf.int32,shape=[batch_size,1])
    #用于验证的词
    valid_dataset=tf.constant(valid_examples,dtype=tf.int32)
    #CPU实现
    with tf.device('/cpu:0'):
        #定义一个embedding变量，相当于一行存储一个词的embedding
        embeddings=tf.Variable(tf.random_uniform([vocabulary_size,embedding_size],-1.0,1.0))
        #利用embedding lookup可以轻松得到一个batch内的所有词的嵌入
        embed=tf.nn.embedding_lookup(embeddings,train_inputs)
        #创建俩变量用于NCE loss
        nce_weights=tf.Variable(tf.truncated_normal([vocabulary_size,embedding_size],stddev=1.0/math.sqrt(embedding_size)))
        nce_biases=tf.Variable(tf.zeros([vocabulary_size]))
        #nce_loss会自动选取噪声词，并且形成损失，随机选取num sampled个噪声词
        loss=tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,biases=nce_biases,labels=train_labels,inputs=embed,num_sampled=num_sampled,num_classes=vocabulary_size))
        #得到loss后构造优化器
        optimizer=tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        #对embedding层做一次归一化
        norm=tf.sqrt(tf.reduce_sum(tf.square(embeddings),1,keep_dims=True))
        normalized_embeddings=embeddings/norm
        #找出和验证词的embedding并计算他们和所有单词的相似度
        valid_embeddings=tf.nn.embedding_lookup(normalized_embeddings,valid_dataset)
        similarity=tf.matmul(valid_embeddings,normalized_embeddings,transpose_b=True)
        #变量初始化
        init=tf.global_variables_initializer()

In [20]:
num_steps=1001
with tf.Session(graph=graph) as session:
    init.run()
    print('initialized')
    average_loss=0
    for step in xrange(num_steps):
        batch_inputs,batch_labels=generate_batch(batch_size,num_skips,skip_window)
        feed_dict={train_inputs:batch_inputs,train_labels:batch_labels}
        #优化一步
        _,loss_val=session.run([optimizer,loss],feed_dict=feed_dict)
        average_loss+=loss
        if step%200==0:
            if step>0:
                average_loss/=200
                print('Average loss at step',step,':',average_loss)
                average_loss=0
        if step%1000==0:
            # sim 是验证词与所有词之间的相似度
            sim=similarity.eval()
            # 一共valid_size个验证词
            for i in xrange(valid_size):
                valid_word=reverse_dictionary[valid_examples[i]]
                top_k=8 #输出最相邻的8个词语
                nearest=(-sim[i,:]).argsort()[1:top_k+1]
                log_str='Nearest to %s:'%valid_word
                for k in xrange(top_k):
                    close_word=reverse_dictionary[nearest[k]]
                    log_str='%s,%s'%(log_str,close_word)
                print(log_str)
    #final_embeddings是最后的embedding向量，它的现状为[vocabulary size，embeddingsize]
    # 每一行代表着对应单词id的词嵌入表示
    final_embeddings=normalized_embeddings.eval()

initialized
Nearest to such:,orlando,syndicate,universiti,sustainability,hesitate,greek,bingen,karnataka
Nearest to all:,domitian,mantua,hearings,softer,divisions,gan,wilfred,backups
Nearest to during:,knopf,loan,colonization,sucks,innes,ducts,yemen,tyrants
Nearest to many:,ailuropoda,preserve,sow,polymers,ruben,elbow,capability,yesterday
Nearest to often:,observation,demarco,fuze,authorize,honneur,subgenres,maladaptive,monmouth
Nearest to at:,stamford,zahir,phimosis,samsara,congress,celebrities,mik,xj
Nearest to also:,geri,shogun,automakers,tech,judeo,brazilians,nanjing,acropolis
Nearest to new:,sit,epinephrine,axioms,cordell,misanthropy,travelers,kazakh,hawkwind
Nearest to zero:,suns,burman,starts,mutagenesis,golda,zemin,badges,abakan
Nearest to called:,cynics,climates,claims,muni,geos,bin,papyrus,drava
Nearest to th:,telharmonium,affecting,les,poses,summands,motivational,petroleum,triune
Nearest to other:,photographers,pharmacist,merseyside,kirlian,regime,immature,nevi,cristobal
Nea

In [23]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
def plot_with_labels(low_dim_embs,labels,filename='tsne.png'):
    assert low_dim_embs.shape[0]>=len(labels),'More labels than embeddings'
    plt.figure(figsize=(18,18)) #in inches
    for i,label in enumerate(labels):
        x,y=low_dim_embs[i,:]
        plt.scatter(x,y)
        plt.annotate(label,xy=(x,y),xytext=(5,2),textcoords='offset points',ha='right',va='bottom')
        plt.savefig(filename)

from sklearn.manifold import TSNE
#t sne 降维
tsne=TSNE(perplexity=30,n_components=2,init='pca',n_iter=5000)
plot_only=500
low_dim_embs=tsne.fit_transform(final_embeddings[:plot_only,:])
labels=[reverse_dictionary[i] for i in xrange(plot_only)]
plot_with_labels(low_dim_embs,labels)

KeyboardInterrupt: 